In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

df = pd.read_csv("data/raw/IMDB Dataset.csv")  # Path to your dataset
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})  # Encode labels

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

test_encodings = tokenizer(
    list(test_texts),
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

train_labels = torch.tensor(train_labels.values)
test_labels = torch.tensor(test_labels.values)

class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDBDataset(train_encodings, train_labels)
test_dataset = IMDBDataset(test_encodings, test_labels)


ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


In [ ]:
import transformers
import accelerate

print(transformers.__version__)  # Should be >= 4.38.0+
print(transformers.__file__)
print(accelerate.__version__)   # Should be >= 0.20.1+


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results/bert',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",      # <-- this should now work!
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none",
)


In [ ]:
def compute_metrics(p):
    preds = torch.argmax(torch.tensor(p.predictions), axis=1)
    labels = torch.tensor(p.label_ids)
    accuracy = (preds == labels).float().mean()
    return {"accuracy": accuracy.item()}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate()
print(f"Test Accuracy: {results['eval_accuracy']:.4f}")